# Intro

This is a one-shot (single cell) notebook that shows curves for a single security over multiple years. 

This depends on `iql` being installed. Run `%pip install iql` if not.

What's shown here:
- Showing tenor and rate for a Curve / curvemembers
- Retrieving historical rates
- Displaying an animated Plotly graph

In [ ]:
# If iql is not installed, uncomment next line
# %pip install iql

import iql
import plotly.express as px
from functools import cache
import pandas as pd
import jinja2

# {{security}} comes from the jinja2 template rendering. 
# pivot=(id, name) just means: put each id on a separate row, and the "name" field (the BQL field name) as the columns
# paramquery is an iql feature to substitute the $OFFSET field with the results of the query

query = """
select *, year(date) as year from bql("
    get(
        id().id_dates as #date,
        id().tenor as #tenor,
        id().year_fraction as #year_fraction,
        rate(side=mid, dates=$OFFSETY) as #rate
        ) 
    for(CURVEMEMBERS('{{security}}', dates=$OFFSETY))
", pivot=(id, name), paramquery=('$OFFSET', 'select * from range(-12, 1)'))
order by year_fraction, year
"""


@cache
def _exec_iql(query: str, **kwargs) -> pd.DataFrame:
    query_string = jinja2.Template(query).render(**kwargs)
    df = iql.execute(query_string)
    return df

curve_df = _exec_iql(query, security='YCGT0001 Index')

    
curve_df = curve_df.sort_values(by=["year_fraction", "year"])


# animation_frame is the thing that the animation will iterate over
fig = px.bar(curve_df, x="year_fraction", y="rate", animation_frame="year", animation_group="year_fraction")

# Using the numerical year_fraction to position the X axis, but show the tenor label
fig.update_xaxes(tickvals=curve_df["year_fraction"], ticktext=curve_df["tenor"])

# Set x and y axes ranges, so the chart is stable when animating
fig.update_yaxes(range=[curve_df["rate"].min(), curve_df["rate"].max()])
fig.update_xaxes(range=[curve_df["year_fraction"].min(), curve_df["year_fraction"].max()])

# Set a fixed bar width, so the bars don't change sizes during animation
fig.update_traces(width=0.5)

fig